# LlamaIndex Async

Async is a growing method for LLM applications and can be especially useful for reducing indexing time. This notebook demonstrates how to monitor Llama-index async apps with TruLens.

### Import from LlamaIndex and TruLens

In [ ]:
# !pip install --pre trulens trulens-instrument-llamaindex trulens-providers-openai llama_index==0.10.11 llama-index-readers-web openai

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
import openai
from trulens.core import Feedback
from trulens.core import TruSession
from trulens.instrument.llamaindex import TruLlama
from trulens.providers.openai import OpenAI

tru = TruSession()

### Add API keys
For this example you need an OpenAI key

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "..."
openai.api_key = os.environ["OPENAI_API_KEY"]

### Create Async App

In [ ]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

In [ ]:
response = query_engine.aquery("What did the author do growing up?")

print(response)  # should be awaitable
print(await response)

### Set up Evaluation

In [ ]:
# Initialize OpenAI-based feedback function collection class:
openai = OpenAI()

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    openai.relevance, name="QA Relevance"
).on_input_output()

### Create tracked app

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine, feedbacks=[f_qa_relevance])

### Run Async Application with TruLens

In [ ]:
with tru_query_engine_recorder as recording:
    response = await query_engine.aquery("What did the author do growing up?")

print(response)

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(tru)